### Exploratory Data Analysis

Before we apply clustering and supervised learning techniques, let's get some idea of the dataset we're working with. First, we import some basic libraries:

In [12]:
# Basic libraries
import numpy as np
import pandas as pd

In [6]:
# Load datasets
steam_df = pd.read_csv('./steam_games.csv',delimiter=';', low_memory=False)
steam_df.head()

,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,Owners,...,Price,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,Valve,Action,"Action: 5426, FPS: 4831, Multiplayer: 3392, Sh...",game,"Multi-player, Valve Anti-Cheat enabled, Online...","10,000,000 .. 20,000,000",...,999,999,0,13990,"English, French, German, Italian, Spanish - Sp...","windows, mac, linux",2000/11/1,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
1,1000000,ASCENXION,ASCENXION is a 2D shoot 'em up game where you ...,IndigoBlue Game Studio,PsychoFlux Entertainment,"Action, Adventure, Indie","""Shoot Em Up"": 186, Metroidvania: 181, Bullet ...",game,"Single-player, Partial Controller Support, Ste...","0 .. 20,000",...,999,999,0,0,"English, Korean, Simplified Chinese",windows,2021/05/14,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
2,1000010,Crown Trick,"Enter a labyrinth that moves as you move, wher...",NEXT Studios,"Team17, NEXT Studios","Adventure, Indie, RPG, Strategy","Rogue-like: 268, Turn-Based Combat: 254, RPG: ...",game,"Single-player, Partial Controller Support, Ste...","200,000 .. 500,000",...,599,1999,70,99,"Simplified Chinese, English, Japanese, Traditi...",windows,2020/10/16,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
3,1000030,"Cook, Serve, Delicious! 3?!","Cook, serve and manage your food truck as you ...",Vertigo Gaming Inc.,Vertigo Gaming Inc.,"Action, Indie, Simulation, Strategy","Typing: 221, Management: 213, Casual: 209, Dif...",game,"Multi-player, Single-player, Co-op, Steam Achi...","100,000 .. 200,000",...,1999,1999,0,76,English,"windows, mac",2020/10/14,0,http://www.cookservedelicious.com,https://cdn.akamai.steamstatic.com/steam/apps/...
4,1000040,细胞战争,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,DoubleC Games,DoubleC Games,"Action, Casual, Indie, Simulation","Action: 22, Casual: 22, Indie: 21, Simulation: 20",game,Single-player,"0 .. 20,000",...,199,199,0,0,Simplified Chinese,windows,2019/03/30,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...


### Bonus: Itch.io Dataset

Initially, we had plans for both a dataset from Steam and a dataset from Itch.io. We later chose to abandon the latter dataset to focus on Steam games, due to some differences between the datasets. For the sake of documentation, however, we are including our initial work on the Itch.io dataset here. First, we read in the dataset to get a sense of what we're working with:

In [9]:
kaggle_df = pd.read_csv('./itch.csv',delimiter=';',encoding = "ISO-8859-1")
kaggle_df.head()

,Name,Description,Price,Author,Rated,RatingNumber,Genre,Link,Platforms,Languages,Tool,Tags
0,Friday Night Funkin',The coolest rhythm game,$0,ninjamuffin99,"94,35021",10011,Rhythm,https://ninja-muffin24.itch.io/funkin,"Windows, macOS, Linux, HTML5",Unknown,"OpenFL, IndieCade, Haxe",2D
1,Sort the Court!,"Give your decree in simple yes or no answers, ...",$0,Graeme Borland,"94,08258",6709,Simulation,https://graebor.itch.io/sort-the-court,"Windows, macOS, Linux, HTML5",English,Unity,"Cute, Ludum Dare 34"
2,We Become What We Behold,"a game about news cycles, vicious cycles, infi...",$0,Nicky Case!,"95,42467",4839,Unknown,https://ncase.itch.io/wbwwb,HTML5,English,PixiJS,"Comedy, Dark Humor, Experimental"
3,HoloCure,"A 'survivors' like, free unofficial Hololive f...",$0,Kay Yu,"98,84398",4775,Action,https://kay-yu.itch.io/holocure,Windows,"English, Japanese",GameMaker: Studio,"Fangame, hololive, Pixel Art, Roguelite, Singl..."
4,?Our Life: Beginnings & Always,Grow from childhood to adulthood with the lone...,$0,GBPatch,"98,92411",4480,Visual Novel,https://gbpatch.itch.io/our-life,"Windows, macOS, Linux, Android",Unknown,Unknown,"amare, Comedy, Dating Sim, Gay, LGBT, Meaningf..."


There are a number of attributes we could work with, but those relevant to the genre or style of a game are mostly discrete. Therefore, we'll start by using the descriptions of each game in a bag-of-words representation, using the default list of stopwords for convenience.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Using base list of stop words
count_vector = CountVectorizer(stop_words="english")

In [14]:
# Preprocessing of kaggle dataset
count_vector.fit(kaggle_df['Description'])
doc_array = count_vector.transform(kaggle_df['Description']).toarray()
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names_out())
d = {}
d['bag_of_words_descripton'] = frequency_matrix
d['info'] = kaggle_df
d['similarity'] = pd.DataFrame(cosine_similarity(frequency_matrix))
kaggle_df = pd.concat(d, axis=1)
kaggle_df.head()

bag_of_words_descripton                                       ...  \
                       00 000 00s 012 04 087 0h 0space 0x72 10  ...   
0                       0   0   0   0  0   0  0      0    0  0  ...   
1                       0   0   0   0  0   0  0      0    0  0  ...   
2                       0   0   0   0  0   0  0      0    0  0  ...   
3                       0   0   0   0  0   0  0      0    0  0  ...   
4                       0   0   0   0  0   0  0      0    0  0  ...   

  similarity                                                              
        9116 9117 9118 9119      9120 9121 9122      9123      9124 9125  
0        0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.288675  0.0  
1        0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.169031  0.000000  0.0  
2        0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.138675  0.0  
3        0.0  0.0  0.0  0.0  0.209657  0.0  0.0  0.000000  0.188982  0.0  
4        0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0  

[5 rows x 18874 columns]

Now that we've preprocessed the data, we can use a basic cosine similarity metric to find the closest descriptions to any given game.

In [15]:
# Takes in a dataframe, an input game, and dictionary keys for
def get_similar_games(df, game_name, info_col, sim_col, thresh):
    temp = df[df[info_col]['Name'] == game_name][sim_col]
    inds = temp.loc[:,((temp > thresh) & (temp < 1.0)).all()]
    t = df.iloc[inds.columns]['info']
    i = inds.melt().set_index('variable')
    t['cosine_sim'] = i
    return t

In [26]:
get_similar_games(kaggle_df, 'Friday Night Funkin\'', 'info', 'similarity', 0.66).sort_values('cosine_sim', ascending=False).head()

,Name,Description,Price,Author,Rated,RatingNumber,Genre,Link,Platforms,Languages,Tool,Tags,cosine_sim
264,Friday Night Funkin' (Ludum Dare Prototype),Probably the coolest rhythm game,$0,ninjamuffin99,"89,44099",483,Rhythm,https://ninja-muffin24.itch.io/friday-night-fu...,"Windows, macOS, HTML5",Unknown,"Haxe, OpenFL","2D, Cute, Ludum Dare, Ludum Dare 47, Music",0.866025
4148,Friday Night Funkin' VR,The coolest rhythm game in first person,$0,Ben Kurtin,"96,42857",28,Rhythm,https://thisisbennyk.itch.io/funkin-vr,Android,English,Godot,"3D, Fangame, Funny, Virtual Reality (VR)",0.866025
6937,Dancing Angel,Rhythm game,$0,snowinrain,"87,27273",11,Rhythm,https://snowinrain.itch.io/dancing-angels,Windows,English,Unity,"3D, Anime, Cute, dancing, Fantasy, Music, musi...",0.816497
3187,ONE PUNCH BEAT (GDKO Round 6),the one button rhythm game!,$0,Federico Calchera,"92,85714",42,Rhythm,https://federico-calchera.itch.io/one-punch-beat,HTML5,Unknown,Construct,"2D, Anime, Arcade, Cyberpunk, Fast-Paced, Musi...",0.666667
4908,BOOBSTAR,JJBA Rhythm Game,$0,Nancy10069,"96,19048",21,Rhythm,https://nancy10069.itch.io/boobstar,HTML5,Unknown,Unity,"2D, Fangame, jjba, jojos-bizarre-adventure",0.666667


In [27]:
get_similar_games(kaggle_df, 'Sort the Court!', 'info', 'similarity', 0.25).sort_values('cosine_sim', ascending=False).head()

,Name,Description,Price,Author,Rated,RatingNumber,Genre,Link,Platforms,Languages,Tool,Tags,cosine_sim
1,Sort the Court!,"Give your decree in simple yes or no answers, ...",$0,Graeme Borland,"94,08258",6709,Simulation,https://graebor.itch.io/sort-the-court,"Windows, macOS, Linux, HTML5",English,Unity,"Cute, Ludum Dare 34",1.000000
4278,soon,simple things become routine and grow from there.,$0,Stuffed Wombat,"96,15385",26,Unknown,https://stuffedwombat.itch.io/soon,HTML5,Unknown,Construct,"3D, First-Person, gameaweek2022, Horror, ritua...",0.377964
575,The Invader,You find yourself in a kingdom do what ever yo...,$0,FadingCometStudios,"96,30769",260,Adventure,https://fadingcometstudios.itch.io/the-invader,"Windows, macOS, Linux",Unknown,Unknown,"3D, Low-poly, Point & Click, Singleplayer, Top...",0.267261
1072,Quit the game to win,It's not as simple as it sounds.,$0,Andrea Pignataro,"81,84211",152,Puzzle,https://approductions.itch.io/quit-the-game-to...,Windows,English,Unknown,"2D, Casual, Comedy, Difficult, Horror, mind-be...",0.267261
4296,Start Powerless,"You can only move at first, but powerups will ...",$0,rob1221,"93,07692",26,Platformer,https://rob1221.itch.io/start-powerless,HTML5,English,"ChipTone, OpenFL, Stencyl, Inkscape","2D, Ludum Dare, Ludum Dare 45, Puzzle-Platform...",0.267261


We can see that results are mixed. For some games, the description contains genre keywords like "rhythm", which allow other games with the same keyword to be easily found by cosine similarity (in this case, it helps that the game is so popular that it has multiple spinoffs). For others, however, the description is not so helpful, as it describes how the game works in simple language with no particular genre-defining keywords.